In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20201229)
tf.set_random_seed(20201229)

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# --- 1段目のコンボリューション層とプーリング層を定義する ---
num_filters1 = 32

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,num_filters1], stddev=0.1))
h_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding='SAME')
b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1) # b_conv1にもreluを適用することでピクセルのカットオフ値もパラメータ最適化の対象とする
h_pool1 =tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [4]:
# --- 2段目のコンボリューション層とプーリング層を定義する ---
num_filters2 = 64

W_conv2 = tf.Variable(tf.truncated_normal([5,5,num_filters1,num_filters2], stddev=0.1))
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding='SAME')
b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2) # b_conv2にもreluを適用することでピクセルのカットオフ値もパラメータ最適化の対象とする
h_pool2 =tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [5]:
# --- 2段目のプーリング層からの出力に対して全結合層を通す ---
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filters2])

num_units1 = 7*7*num_filters2
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) + b2)

keep_prob = tf.placeholder(tf.float32) # dropoutの割合を設定
hidden2_drop = tf.nn.dropout(hidden2, keep_prob) # dropoutを設定

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2_drop, w0) + b0)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
# --- 損失関数を定義 ---
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
# --- パラメータを最適化 ---
sess = tf.Session()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver()
i = 0
for _ in range(20000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step, feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
    if i % 500 == 0:
        loss_vals, acc_vals = [], []
        for c in range(4):
            start = int(len(mnist.test.labels) / 4) * c
            end = int(len(mnist.test.labels) / 4) * (c + 1)
            loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x:mnist.test.images[start:end], t:mnist.test.labels[start:end], keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print ('Step: %d, Loss: %f, Accuracy: %f' % (i, loss_val, acc_val))
        saver.save(sess, 'cnn_session', global_step=i)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Step: 500, Loss: 1458.870483, Accuracy: 0.958600
Step: 1000, Loss: 975.812012, Accuracy: 0.971500
Step: 1500, Loss: 807.803772, Accuracy: 0.975600
Step: 2000, Loss: 641.931152, Accuracy: 0.979800
Step: 2500, Loss: 579.376221, Accuracy: 0.981000
Step: 3000, Loss: 555.310913, Accuracy: 0.982800
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Step: 3500, Loss: 479.356567, Accuracy: 0.983900
Step: 4000, Loss: 514.845093, Accuracy: 0.982800
Step: 4500, Loss: 463.100861, Accuracy: 0.984300
Step: 5000, Loss: 454.873779, Accuracy: 0.984700
Step: 5500, Loss: 418.310608, Accuracy: 0.986100
Step: 6000, Loss: 370.734711, Accuracy: 0.987200
Step: 6500, Loss: 414.933289, Accuracy: 0.984400
Step: 7000, Loss: 395.302551, Accuracy: 0.985600
Step: 7500, Loss: 385.224640, Accuracy: 0.985800
Step: 8000, Loss: 357.374725, Accuracy: 0.987900
Step: 8500, Loss: 379.930206, Accuracy: 0.987200
Step: 900